In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Meu drive pessoal - Variaveis Globais

In [ ]:
id_grupo = 1
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/rn/grupos_pagamentos/grupo_{id_grupo}/pagamentos_grupo_{id_grupo}"
path_drive_csv = f"/content/drive/MyDrive/DataBase/world_bank/rn/grupos_pagamentos/grupo_{id_grupo}/links_coletados_pagamentos_grupo_{id_grupo}"
local_path_csv = f"/content/links_coleta_grupo/links_coletados_pagamentos_grupo_{id_grupo}.csv"

# Drive Compartilhado - Variaveis Globais

In [ ]:
id_grupo = 1
categoria = "pagamentos"
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/ComprasPublicas_Brasil/input/RN/grupos_pagamento/grupo_{id_grupo}/{categoria}_grupo_{id_grupo}"
path_drive_csv = f"/content/drive/MyDrive/DataBase/world_bank/ComprasPublicas_Brasil/input/RN/grupos_pagamento/grupo_{id_grupo}/links_coletados_{categoria}_grupo_{id_grupo}"
local_path_csv = f"/content/links_coleta_grupo/links_coletados_pagamentos_grupo_{id_grupo}.csv"

# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '35.188.236.202', 'country': 'United States', 'cc': 'US'}

# Importação e função

In [ ]:
import requests
from urllib3 import PoolManager
import os
import csv
import shutil
import glob
import concurrent.futures
import datetime as dt
import time
import re
import pandas as pd
from time import sleep
from more_itertools import batched
import asyncio
import aiohttp
from aiohttp.client import ClientSession
import nest_asyncio


def csv_manager(path: str, rows: list, mode: str, fieldnames: list) -> None:

      with open(path, mode, newline='', encoding='utf-8') as file:
          # Create a CSV writer object
          writer = csv.DictWriter(file, fieldnames=fieldnames)

          # Write the field names
          if mode == 'w':
              writer.writeheader()
          writer.writerows(rows)

def is_duplicate(path: str, row: dict) -> bool:
  csv = csv_read(path)
  for key, value in row.items():
    if value == None:
      row[key] = ""
      continue
    row[key] = str(value)
  return row in csv

def csv_read(path: str) -> list:
  with open(path, 'r', encoding="utf-8") as file:
    existing_data = []
    dict_reader = csv.DictReader(file)
    for row in dict_reader:
        existing_data.append(row)
    return existing_data

def empenhos_api(ano: int, identificador: int|str) -> str:

  url = f"https://apidadosabertos.tce.rn.gov.br/api/EmpenhosLiquidacoesPagamentosApi/Empenhos/Json/{ano}/{identificador}"
  # print(f"Request para: {url}")
  # data = requests.get(url).json()

  return url

def liquidacoes_api(classificacaoInstitucional: str,
                    dataEmpenho: str,
                    numeroEmpenho: str,
                    identificadorUnidade: int|str,
                    formato: str = "json") -> str:

  base_url = "https://apidadosabertos.tce.rn.gov.br/api/EmpenhosLiquidacoesPagamentosApi/Liquidacoes/"
  parameters = f"{formato}/{classificacaoInstitucional}/{dataEmpenho}/{numeroEmpenho}/{identificadorUnidade}"
  url = base_url + parameters

  return url

def pagamentos_api(classificacaoInstitucional: str|int,
                    dataEmpenho: str,
                    numeroEmpenho: str,
                    identificadorUnidade: int,
                    dataLiquidacao: str,
                    numeroLiquidacao: str,
                    formato: str = "json") -> str:

  base_url = "https://apidadosabertos.tce.rn.gov.br/api/EmpenhosLiquidacoesPagamentosApi/Pagamentos/"

  parameters = f"{formato}/{classificacaoInstitucional}/{dataEmpenho}/{numeroEmpenho}/{dataLiquidacao}/{numeroLiquidacao}/{identificadorUnidade}"
  url = base_url + parameters

  return url

def coleted_list(path_input: str = "/content/input") -> list:
  path_verify = f"{path_input}/**/*/*/"
  csv_files_verify = glob.glob(path_verify, recursive=True)
  ids = [folder.split("/")[-2] for folder in csv_files_verify]
  ids = list(dict.fromkeys(ids))
  return ids


def save_data(data: dict) -> None:

  date = data["dataEmpenho"].split("-")
  ano = date[0]
  mes = str(date[1]).lstrip("0")
  columns = list(data.keys())
  csv_file_name = f"empenhos_{codigo_orgao}_{mes}_{ano}.csv"
  path_csv_folder = os.path.join(base_path, ano,
              str(identificador_unidade))
  path_csv = os.path.join(path_csv_folder, csv_file_name)

  if not os.path.exists(path_csv_folder) or not os.path.exists(path_csv):
    os.makedirs(path_csv_folder, exist_ok=True)
    csv_manager(path_csv, [data], "w", columns)
  else:
    csv_manager(path_csv, [data], "a", columns)

def get_year_json(link: str, year: int) -> tuple:
  json = http.request('GET', link).json()
  return (year, json)

def make_list_chunks(values: list, chunk_size:int = 2000) -> list:
  resquet_in_chunks = [chunk for chunk in batched(values, chunk_size)]
  return resquet_in_chunks

# Extrair

In [ ]:
from zipfile import ZipFile

def extrair(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)

In [ ]:
liqui = f"/content/drive/MyDrive/DataBase/world_bank/ComprasPublicas_Brasil/input/RN/Liquidação/liquidacao_rn.zip"

In [ ]:
extrair(path=liqui,
        path_output="/content/liqui/")

## Extrair CSV

In [ ]:
extrair(path=path_drive_csv + '.zip',
        path_output="/content/links_coleta_grupo")

In [ ]:
extrair(path=path_drive_input + '.zip',
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
extrair(path=path_drive_csv + "_auto" + '.zip',
        path_output="/content/links_coleta_grupo")

In [ ]:
extrair(path=path_drive_input + "_auto" + '.zip',
        path_output="/content/input")

# Raspagem

In [ ]:
async def download_link(link: str, session: ClientSession) -> list:

    # try:
    async with session.get(link) as response:

            result = await response.json()
            return [link, result]

    # except Exception as erro:
    #     return erro

async def download_all(links: list):
    my_conn = aiohttp.TCPConnector(limit=20, ssl=False)
    # timeout = aiohttp.ClientTimeout(total=20)
    async with aiohttp.ClientSession(connector=my_conn) as session:
        slots = []
        for link in links:
            slot = asyncio.ensure_future(download_link(link=link, session=session))
            slots.append(slot)
        await asyncio.gather(*slots, return_exceptions=True)# the await must be nest inside of the session
    return slots


def save_data_pagamento(data: dict, link: str) -> str|None:

  try:

    print(data)
    id_unidade = link.split("/")[-1]
    data["identificador_unidade"] = id_unidade
    date = data["dataPagamento"].split("-")
    ano = date[0]
    mes = str(date[1]).lstrip("0")
    columns = list(data.keys())
    csv_file_name = f"pagamento_{mes}_{ano}.csv"
    path_csv_folder = os.path.join(base_path, ano,
                str(id_unidade))
    path_csv = os.path.join(path_csv_folder, csv_file_name)

    if not os.path.exists(path_csv_folder) or not os.path.exists(path_csv):
      os.makedirs(path_csv_folder, exist_ok=True)
      csv_manager(path_csv, [data], "w", columns)
    else:
      csv_manager(path_csv, [data], "a", columns)

    return link

  except Exception as error:

    print(f"{'--' * 50}\nErro: {error}\n{data}\n{'--' * 50}")

def get_csvs_from_id(id_unidade: str|int) -> list:
  csv_orgao = f"/content/input_empenho/**/{id_unidade}/*.csv"
  csvs = glob.glob(csv_orgao)
  return csvs

def get_json(link: str) -> list:

  data_json = "Vazio"
  response = ""

  try:

    response = http.request('GET', link)

    if 400 <= response.status < 600:
      raise Exception(f'Request failed with status code {response.status}')

    data_json = response.json()

  except Exception as error_get_json:
    print("*-*" * 30)
    print(f"Erro: {error_get_json}\nFalha ao consegui o JSON")
    print(response.status_code)
    print(response.text)
    print(response.content)
    print("*-*" * 30)
  finally:

    return [link, data_json]

def extract(links: list) -> list|dict:

  coletados = []
  tasks = asyncio.run(download_all(links))

  datas = [slot.result() for slot in tasks if not slot.exception()]
  datas = {link: json for link, json in datas if type(json) == list}
  # datas = {key: data for key, data in datas.items() if type(data) == list}

  for key, data in datas.items():
    coletados = list(map(lambda data: save_data_pagamento(data, key), data))

  print(coletados)
  # return coletados
  return list(datas.keys())

def atualizar_df_coletados(df_coletados: pd.DataFrame, links: list) -> pd.DataFrame:
  marks = df_coletados.link.isin(links)
  df_coletados.loc[marks, ["coletado"]] = True
  return df_coletados


def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def send_solo_drive() -> None:

  drive.mount('/content/drive')

  send_folder_drive(path_drive_input + "_auto",
                    "/content/input/")

  send_folder_drive(path_drive_csv + "_auto",
                    "/content/links_coleta_grupo")


In [ ]:
df_coletados = pd.read_csv(local_path_csv)

In [ ]:
nest_asyncio.apply()
base_path = os.path.join(os.getcwd(), "input")
para_coletar = df_coletados[df_coletados.coletado == False].link.values
links_chunk = make_list_chunks(para_coletar, chunk_size=500)

for n, chunk in enumerate(links_chunk):

  try:
    start_time = time.time()
    print(f"Chunk: {n+1}/{len(links_chunk)} Iniciado\n")
    links_coletados = extract(chunk)
    print(f"Foram Coletados: {len(links_coletados)}")
    df_coletados = atualizar_df_coletados(df_coletados, links_coletados)
    print(f"Duração da execução: {(time.time() - start_time) / 60:.2f}\n{'-' * 30}")
    sleep(20)
    break

    # if (n + 1) % 10 == 0:
    if len(links_coletados) == 0:
      break
    #   df_coletados.to_csv(local_path_csv, index=False)
    #   send_solo_drive()

  except Exception as erro:
    print(f"Erro: {erro}\nFalha chunk {n+1}")
    df_coletados.to_csv(local_path_csv, index=False)
    sleep(30)


# send_solo_drive()

Chunk: 1/401 Iniciado

[]
Foram Coletados: 500
Duração da execução: 0.14
------------------------------


In [ ]:
df_coletados.loc[20000].values

array(['https://apidadosabertos.tce.rn.gov.br/api/EmpenhosLiquidacoesPagamentosApi/Pagamentos/json/04100/2020-02-03/04100203001/2020-05-28/04100146/390',
       False, nan, 1], dtype=object)

In [ ]:
para_coletar = df_coletados[df_coletados.coletado == False].link.values
links_chunk = make_list_chunks(para_coletar, chunk_size=1000)
len(links_chunk)

201

# Verificar Dados corretos

In [ ]:
import csv

def ler_csv(path: str = "/content/verificar_files.csv") -> list:

  with open(path, "r") as txt_csv:

    csv_rows = csv.DictReader(txt_csv)
    rows = [row for row in csv_rows]
    return rows


def verificar_links(rows: list, link: str) -> int:
  sum = 0
  arg = link.split("/")[-4::]
  for row in rows:
    if row.get("classificacaoInstitucional") == str(arg[0]) and row.get("dataEmpenho") == arg[1] and row.get("numeroEmpenho") == arg[2] and row.get("identificador_unidade") == str(arg[3]):
      sum += 1

  # mark = df_verifica[(df_verifica.classificacaoInstitucional == str(arg[0])) &
  #  (df_verifica.dataEmpenho == arg[1]) &
  #   (df_verifica.numeroEmpenho == arg[2]) &
  #    (df_verifica.identificador_unidade == str(arg[3]))]
  # return mark.shape[0]
  return sum


def criar_key(link: str) -> str:

  arg = link.split("/")[-4::]

  key = "".join([str(arg[0]), arg[1],
    arg[2], str(arg[3])])

  return key


def contar_encontros(row: pd.DataFrame, credibilidade: dict):

  key = "".join([str(row.classificacaoInstitucional), row.dataEmpenho,
    row.numeroEmpenho, str(row.identificador_unidade)])

  try:
    credibilidade[key][1] += 1
  except:
    pass

  return credibilidade



In [ ]:
path_verify = f"/content/input2/**/*.csv"
csv_files_verify = glob.glob(path_verify, recursive=True)

df_verifica = pd.concat([pd.read_csv(csv_file, dtype=str) for csv_file in csv_files_verify])
links_coletados = df_coletados[df_coletados.coletado == True].link.values
credibilidade = {criar_key(link): [link, 0] for link in links_coletados}

for row in df_verifica.itertuples():
  credibilidade = contar_encontros(row, credibilidade)

varios = {value[0]:value[1] for key, value in credibilidade.items() if value[1] == 0}
encontrado = {value[0]:value[1] for key, value in credibilidade.items() if value[1] > 0}

print(f"Foram coletados: {len(links_coletados)} links\n" \
      f"Numero de dados vinculados a um link: {len(encontrado)}\n" \
      f"Numero de dados possivelmente vazios e não vinculados: {len(varios)}\n")

Foram coletados: 198264 links
Numero de dados vinculados a um link: 78117
Numero de dados possivelmente vazios e não vinculados: 120147



## Verificar vazios

In [ ]:
list(varios.keys())[-10]

'https://apidadosabertos.tce.rn.gov.br/api/EmpenhosLiquidacoesPagamentosApi/Liquidacoes/json/nan/2018-02-20/070016/454'

# Liquidação ta certo

# Mandar para o drive Manualmente

In [ ]:
drive.mount('/content/drive')

df_coletados.to_csv(local_path_csv, index=False)

send_folder_drive(path_drive_input,
                "/content/input/")

send_folder_drive(path_drive_csv,
                "/content/links_coleta_grupo")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
